---
modules

In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd

from keplergl import KeplerGl
import h3
# import googlemaps

---
creating datasets

bike theft data  

[Website zum Datensatz daten.berlin.de](https://daten.berlin.de/datensaetze/fahrraddiebstahl-berlin)

- Lizenz:
    - Creative Commons Namensnennung CC-BY License
- Kategorie:
   - Öffentliche Sicherheit
- Geographische Abdeckung: 
    - Berlin
- Geographische Granularität: 
    - Berlin
- Zeitliche Granularität: 
    - Tag
- Veröffentlicht: 
    - 08.09.2021
- Aktualisiert: 
    - 09.09.2021
- Veröffentlichende Stelle: 
    - Polizei Berlin LKA St 14
- E-Mail Kontakt: 
    - onlineredaktion AT polizei.berlin.de

In [2]:
# df = pd.read_csv('data/Fahrraddiebstahl.csv') # not running, encoding must be set!
df = pd.read_csv('data/Fahrraddiebstahl.csv', encoding='latin-1')
df.columns = df.columns.str.lower()
df.head(2)

,angelegt_am,tatzeit_anfang_datum,tatzeit_anfang_stunde,tatzeit_ende_datum,tatzeit_ende_stunde,lor,schadenshoehe,versuch,art_des_fahrrads,delikt,erfassungsgrund
0,14.09.2020,10.09.2020,10,10.09.2020,12,3400723,706,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern
1,29.09.2020,09.09.2020,16,10.09.2020,7,9200716,220,Nein,Damenfahrrad,Fahrraddiebstahl,Sonstiger schwerer Diebstahl von Fahrrädern


In [3]:
df['lor_str'] = df['lor'].map(lambda x: (str(x)))
df['lor_zf'] = df['lor_str'].apply(lambda x: x.zfill(8)) #lor zero fill
df['plr_id'] = df['lor_zf'].map(lambda x: (x))
df['bzr_id'] = df['lor_zf'].map(lambda x: (x)[-6:])
df['pgr_id'] = df['lor_zf'].map(lambda x: (x)[-4:])

df.drop(columns='lor_str', inplace=True)

df.columns

Index(['angelegt_am', 'tatzeit_anfang_datum', 'tatzeit_anfang_stunde',
       'tatzeit_ende_datum', 'tatzeit_ende_stunde', 'lor', 'schadenshoehe',
       'versuch', 'art_des_fahrrads', 'delikt', 'erfassungsgrund', 'lor_zf',
       'plr_id', 'bzr_id', 'pgr_id'],
      dtype='object')

LOR shapefile to geopandas dataframe

_data/LOR_SHP_2021/lor_plr.shp_  
_data/LOR_SHP_2021/lor_bzr.shp_  
_data/LOR_SHP_2021/lor_pgr.shp_  

PLR: 8 digits, most granular  
BZR: 6 digits  
PGR: 4 digits

In [4]:
gdf = gpd.GeoDataFrame.from_file('data/LOR_SHP_2021/lor_plr.shp') #.replace({-999: None})
gdf.columns = gdf.columns.str.lower()

In [5]:
#transforms dataframe to new crs - coordinate reference system, here utm to wgs/latlong
gdf_wgs = gdf.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [6]:
# creating centroids of polygons as gpd point wkt
gdf_wgs['points'] = gdf_wgs['geometry'].centroid

# extracting lat and long from the point geometry
gdf_wgs["lat"] = gdf_wgs.points.y
gdf_wgs["lon"] = gdf_wgs.points.x


# drop point geometry to not confuse kepler (to gdf geometry columns are misleading)
gdf_wgs.drop(columns='points', inplace=True)

In [7]:
gdf_wgs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   plr_id    542 non-null    object  
 1   plr_name  542 non-null    object  
 2   geometry  542 non-null    geometry
 3   lat       542 non-null    float64 
 4   lon       542 non-null    float64 
dtypes: float64(2), geometry(1), object(2)
memory usage: 21.3+ KB


---
plotting shapes/centroids in kepler

In [41]:
map_1 = KeplerGl(height=800) #create an instance of this class

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [42]:
map_1.add_data(data=gdf_wgs, name='LOR')

In [43]:
map_1

KeplerGl(data={'LOR': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,…

---
merging

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39407 entries, 0 to 39406
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   angelegt_am            39407 non-null  object
 1   tatzeit_anfang_datum   39407 non-null  object
 2   tatzeit_anfang_stunde  39407 non-null  int64 
 3   tatzeit_ende_datum     39407 non-null  object
 4   tatzeit_ende_stunde    39407 non-null  int64 
 5   lor                    39407 non-null  int64 
 6   schadenshoehe          39407 non-null  int64 
 7   versuch                39407 non-null  object
 8   art_des_fahrrads       39407 non-null  object
 9   delikt                 39407 non-null  object
 10  erfassungsgrund        39407 non-null  object
 11  lor_zf                 39407 non-null  object
 12  plr_id                 39407 non-null  object
 13  bzr_id                 39407 non-null  object
 14  pgr_id                 39407 non-null  object
dtypes: int64(4), object

In [12]:
gdf_wgs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   plr_id    542 non-null    object  
 1   plr_name  542 non-null    object  
 2   geometry  542 non-null    geometry
 3   lat       542 non-null    float64 
 4   lon       542 non-null    float64 
dtypes: float64(2), geometry(1), object(2)
memory usage: 21.3+ KB


In [13]:
df_combined = gdf_wgs.merge(df, on='plr_id')
df_combined.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 39407 entries, 0 to 39406
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   plr_id                 39407 non-null  object  
 1   plr_name               39407 non-null  object  
 2   geometry               39407 non-null  geometry
 3   lat                    39407 non-null  float64 
 4   lon                    39407 non-null  float64 
 5   angelegt_am            39407 non-null  object  
 6   tatzeit_anfang_datum   39407 non-null  object  
 7   tatzeit_anfang_stunde  39407 non-null  int64   
 8   tatzeit_ende_datum     39407 non-null  object  
 9   tatzeit_ende_stunde    39407 non-null  int64   
 10  lor                    39407 non-null  int64   
 11  schadenshoehe          39407 non-null  int64   
 12  versuch                39407 non-null  object  
 13  art_des_fahrrads       39407 non-null  object  
 14  delikt                 39407 n

In [14]:
df_combined.dropna(axis=0, inplace=True)
df_combined.shape

(39407, 19)

In [15]:
type(df_combined.geometry[0])

shapely.geometry.polygon.Polygon

---
plotting df_combined

39.407 observations: rendering interrupted aftzer 6 minutes  
1.000 observations: map not zoomable, loses shapes  

so ... creating a grouped dataframe counting the thefts

In [16]:
df_plr_group = df.groupby(by='plr_id').count().reset_index()
df_plr_group = df_plr_group[['plr_id', 'delikt']]
df_plr_group.head()

,plr_id,delikt
0,01100101,71
1,01100102,148
2,01100103,118
3,01100104,103
4,01100205,43


In [60]:
df_combined = gdf_wgs.merge(df_plr_group, on='plr_id')
df_combined.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 540 entries, 0 to 539
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   plr_id    540 non-null    object  
 1   plr_name  540 non-null    object  
 2   geometry  540 non-null    geometry
 3   lat       540 non-null    float64 
 4   lon       540 non-null    float64 
 5   delikt    540 non-null    int64   
dtypes: float64(2), geometry(1), int64(1), object(2)
memory usage: 29.5+ KB


adding this leads to whitescreen, wtf?!

In [61]:
# adding h3 hexbin h3_id
# df_combined['hex'] = df_combined.apply(lambda x: h3.geo_to_h3(x.lat, x.lon, resolution = 7), axis = 1)

In [62]:
df_combined.hex.unique

AttributeError: 'GeoDataFrame' object has no attribute 'hex'

In [19]:
df_combined.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 540 entries, 0 to 539
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   plr_id    540 non-null    object  
 1   plr_name  540 non-null    object  
 2   geometry  540 non-null    geometry
 3   lat       540 non-null    float64 
 4   lon       540 non-null    float64 
 5   delikt    540 non-null    int64   
 6   hex       540 non-null    object  
dtypes: float64(2), geometry(1), int64(1), object(3)
memory usage: 33.8+ KB


In [67]:
map_2 = KeplerGl(height=800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [68]:
map_2.add_data(data=df_combined, name='PLR')

In [69]:
map_2

KeplerGl(data={'PLR': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,…

In [71]:
map_conf = map_2.config

In [74]:
map_conf

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'ffw53pr',
     'type': 'point',
     'config': {'dataId': 'PLR',
      'label': 'Point',
      'color': [246, 209, 138],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'lat', 'lng': 'lon', 'altitude': None},
      'isVisible': False,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange

In [77]:
map_4 = KeplerGl(height=800, data={'PLR': df_combined}, config=map_conf)
# map_4.add_data(data=df_combined, name='PLR')
map_4

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ffw53pr', 'type': …

---
H3

In [50]:
df_hex = df_combined[['hex', 'delikt']]
# df_hex.rename(columns={'hex' : 'geometry'}, inplace=True)

In [51]:
df_hex.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540 entries, 0 to 539
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hex     540 non-null    object
 1   delikt  540 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.7+ KB


In [52]:
map_3 = KeplerGl(height=800)


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [53]:
map_3.add_data(data=df_hex, name='H3')
map_3

KeplerGl(data={'H3': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, …

---
lat long